# Data 100 Preprocessing & Modeling Reference

This notebook is a short reference for using scikit-learn as it is presented in Data 100.

In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

## Linear Regression: MPG



To study linear regression, we will use the MPG dataset that comes with seaborn.

In [37]:
mpg = sns.load_dataset("mpg")
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


### Preprocessing

There are three main tasks we need to take care of in preprocessing our data:
1. Dropping unusable columns
2. One-hot encoding non-quantitative variables
3. Handing `NaN` values

Recall that we can use `pd.get_dummies` to one-hot encode a subset of columns in a dataframe, and that we set `drop_first=True` as the first value of each variable is accounted for in the bias term of the regression.

In [132]:
pd.get_dummies(mpg, columns=["origin"], drop_first=True)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,name,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,chevrolet chevelle malibu,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,buick skylark 320,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,plymouth satellite,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,amc rebel sst,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,ford torino,0,1
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,ford mustang gl,0,1
394,44.0,4,97.0,52.0,2130,24.6,82,vw pickup,0,0
395,32.0,4,135.0,84.0,2295,11.6,82,dodge rampage,0,1
396,28.0,4,120.0,79.0,2625,18.6,82,ford ranger,0,1


Now let's take a look at the `NaN` values in each column.

In [45]:
mpg.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

Because there are only 6 rows and all in one column, we will just drop the `NaN` rows. The function `preprocess_mpg` accomplishes this preprocessing pipeline for us.

In [134]:
def preprocess_mpg(data):
    data = pd.get_dummies(data, columns=["origin"], drop_first=True)
    data = data.drop("name", axis=1)
    data = data.dropna()
    X = data.drop("mpg", axis=1).values
    y = data["mpg"].values
    return X, y

Finally, we split our data into training and testing sets using [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split).

In [133]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(mpg, random_state=42)
X_train, y_train = preprocess_mpg(train)
X_test, y_test = preprocess_mpg(test)

### Modeling

Linear regression in sklearn is performed by the [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) object. We leave the `fit_intercept` parameter to its default value `True` so that sklearn automatically adds the bias term for us.

In [138]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_fitted = model.predict(X_train)
y_pred = model.predict(X_test)

### Metrics

sklearn comes with several ways to quantify the ability of our model to accurately predict the real world. We primarily use RMSE and MAE in this course, both of which can be gotten from sklearn. The former is provided in [`sklearn.metrics.mean_squared_error`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error).

In [141]:
from sklearn.metrics import mean_squared_error

print("Training MSE: %.5f" % mean_squared_error(y_train, y_fitted))
print("Testing MSE: %.5f" % mean_squared_error(y_test, y_pred))

Training MSE: 11.37691
Testing MSE: 8.90776


Note, however, that sklearn only provides MSE, rather than **R**MSE. To get the RMSE, we need to take the square root of the MSE.

In [142]:
print("Training RMSE: %.5f" % np.sqrt(mean_squared_error(y_train, y_fitted)))
print("Testing RMSE: %.5f" % np.sqrt(mean_squared_error(y_test, y_pred)))

Training RMSE: 3.37297
Testing RMSE: 2.98459


Finally, we get the MAE using [`sklearn.metrics.mean_absolute_error`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error).

In [143]:
from sklearn.metrics import mean_absolute_error

print("Training MAE: %.5f" % mean_absolute_error(y_train, y_fitted))
print("Testing MAE: %.5f" % mean_absolute_error(y_test, y_pred))

Training MAE: 2.58159
Testing MAE: 2.34989


### Cross Validation

We can perform k-fold cross validation using [`sklearn.model_selection.KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) to split the data. Note that the `KFold` object takes 3 arguments: the number of splits `n_splits`, whether or not to shuffle `shuffle`, and a random seed `random_state`.

In [66]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
errors = []
for train_idx, valid_idx in kfold.split(X_train):
    model = LinearRegression()
    model.fit(X_train[train_idx], y_train[train_idx])
    y_pred = model.predict(X_train[valid_idx])
    errors.append(mean_squared_error(y_train[valid_idx], y_pred))
    
print("3-Fold CV RMSE: %.3f (%.3f)" % (np.mean(np.sqrt(errors)), np.std(np.sqrt(errors))))

3-Fold CV RMSE: 3.495 (0.142)


We can get out of writing all this messy code by using the cross validation that sklearn automates in [`sklearn.model_selection.cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score). This function performs cross validation using the native `.score` method of the estimator passed to it, and we can control the number of folds using the `cv` parameter.

In [70]:
from sklearn.model_selection import cross_val_score

cross_val_score(LinearRegression(), X_train, y_train, cv=3)

array([0.82308114, 0.78382358, 0.78845381])

If we want more control over the folds than just the number, we can also pass a `KFold` instance to `cv`. For example, the code below also shuffles the data before creating the splits.

In [71]:
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
cross_val_score(LinearRegression(), X_train, y_train, cv=kfold)

array([0.80471079, 0.80995701, 0.78289372])

### Ridge Regression

We can also perform ridge regression (linear regression with L2 regularization) using the [`sklearn.linear_model.Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) object.

In [144]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(X_train, y_train)
y_fitted = model.predict(X_train)
y_pred = model.predict(X_test)

print("Training RMSE: %.3f" % np.sqrt(mean_squared_error(y_train, y_fitted)))
print("Testing RMSE: %.5f" % np.sqrt(mean_squared_error(y_test, y_pred)))

Training RMSE: 3.373
Testing RMSE: 2.98342


#### Hyperparameter Tuning

To tune the regularization parameter $\alpha$ of our ridge regression, we use k-fold cross-validation, as demonstrated below.

In [86]:
alphas = np.geomspace(1e-9, 1e3, 15)

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
errors = []
for alpha in alphas:
    alpha_errors = []
    for train_idx, valid_idx in kfold.split(X_train):
        model = Ridge(alpha=alpha)
        model.fit(X_train[train_idx], y_train[train_idx])
        y_pred = model.predict(X_train[valid_idx])
        alpha_errors.append(np.sqrt(mean_squared_error(y_train[valid_idx], y_pred)))
    errors.append(np.mean(alpha_errors))
    
best_alpha_idx = np.argmin(errors)
best_alpha = alphas[best_alpha_idx]
print("Alpha = %s RMSE: %.3f" % (best_alpha, errors[best_alpha_idx]))

Alpha = 1e-09 RMSE: 3.495


We can cut some of the code on this process again by using the [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) object that performs cross-validation to find the best parameters of a model.

In [91]:
from sklearn.model_selection import GridSearchCV

params = {
    "alpha": np.geomspace(1e-9, 1e3, 15)
}
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
cv = GridSearchCV(Ridge(), params, cv=kfold)
cv.fit(X_train, y_train)
cv.best_estimator_

Ridge(alpha=1e-09, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

Since we got the lowest value in our hyperparameter values, we should continue going smaller until we find the best performing value.

In [93]:
params = {
    "alpha": np.geomspace(1e-16, 1e-9, 15)
}
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
cv = GridSearchCV(Ridge(), params, cv=kfold)
cv.fit(X_train, y_train)
cv.best_estimator_

Ridge(alpha=1e-13, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

## Logistic Regression: Titanic

To look at logistic regression, we will us the Titanic dataset that comes with seaborn.

In [29]:
titanic = sns.load_dataset("titanic")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


### Preprocessing

We see that this dataset has a lot more `NaN`s than the MPG dataset. To fix this, we will drop the rows with `NaN`s in `age` and `embarked`. However, since `deck` has so many `NaN`s compared to the size of the dataset, we will simply remove that column from consideration.

In [94]:
titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

We can 

In [114]:
from scipy.stats import mode

def interpolate(values, fn=lambda v: mode(v)[0]):
    values = np.copy(values)
    val = fn(values[~np.isnan(values)])
    np.place(values, np.isnan(values), val)
    return values

interpolate(np.array([2, 2, 1, 3, 4, 5, np.nan])), interpolate(np.array([2, 2, 1, 3, 4, 5, np.nan]), fn=np.median)

(array([2., 2., 1., 3., 4., 5., 2.]),
 array([2. , 2. , 1. , 3. , 4. , 5. , 2.5]))

In [118]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [119]:
def preprocess_titanic(data):
    na_cols = ["age", "embarked"]
    ohe_cols = ["pclass", "sex", "sibsp", "parch", "embarked", "class", "who", "adult_male", "alone"]
    drop_cols = ["deck", "embark_town", "alive"]
    
    data = data.dropna(subset=na_cols)
    data = data.drop(drop_cols, axis=1)
    data = pd.get_dummies(data, columns=ohe_cols, drop_first=True)
    
    X = data.drop("survived", axis=1).values
    y = data["survived"].values
    return X, y

In [120]:
train, test = train_test_split(titanic, random_state=42)
X_train, y_train = preprocess_titanic(train)
X_test, y_test = preprocess_titanic(test)

### Modeling

In [126]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_fitted = model.predict(X_train)

### Metrics

In [128]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_fitted)

0.8258426966292135

### Cross Validation

In [131]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
errors = []
for train_idx, valid_idx in kfold.split(X_train):
    model = LogisticRegression()
    model.fit(X_train[train_idx], y_train[train_idx])
    y_pred = model.predict(X_train[valid_idx])
    errors.append(accuracy_score(y_train[valid_idx], y_pred))
    
print("3-Fold CV RMSE: %.3f (%.3f)" % (np.mean(np.sqrt(errors)), np.std(np.sqrt(errors))))

3-Fold CV RMSE: 0.898 (0.024)
